In [1]:
import sqlite3
conn =sqlite3.connect('api.db')

In [ ]:
import sqlite3
import requests

conn = sqlite3.connect("AQI.db")
cursor = conn.cursor()
create_table_sql = '''
CREATE TABLE IF NOT EXISTS records (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    sitename TEXT NOT NULL,
    county TEXT,
    aqi INTEGER,
    status TEXT,
    pm25 NUMERIC,
    date TEXT,
    lat NUMERIC,
    lon NUMERIC,
    UNIQUE(sitename, date) 
);
'''
cursor.execute(create_table_sql)
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate%20desc&format=JSON'
response = requests.get(url)
data = response.json()
sitenames = []

for items in data['records']:
    sitenames.append((
        items['sitename'],
        items['county'],
        items['aqi'],
        items['status'],                      
        items['pm2.5'],
        items['datacreationdate'],            
        items['latitude'],
        items['longitude']
    ))

insertSQL = '''
INSERT INTO records (sitename, county, aqi, status, pm25, date, lat, lon)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
'''
cursor.executemany(insertSQL, sitenames)

conn.commit()
cursor.close()
conn.close()